# Validação da qualidade dos dados brutos
Execução de scripts que validem a qualidade dos dados retornados por todos os Bronzes.

In [0]:
#Importando as bibliotecas necessárias

from pyspark.sql.functions import col, count, when, isnan, length, regexp_replace

In [0]:
# Carregamento das tabelas necessárias e disponíveis nos arquivos bronze
cvm_bruto = spark.table("cvm_bruto")
brasil_cnpj = spark.table("cnpj_bruto")
brasil_uf   = spark.table("uf_bruto")

def qualidadeDados(df, fonte, range_checks=None):
    # Contagem do total de linhas
    total_rows = df.count()
    
    # Cálculo do % de nulos por coluna 
    nulls = []
    for c in df.columns:
        null_count = df.filter(col(c).isNull()).count()
        null_ratio = null_count / total_rows if total_rows > 0 else 0
        nulls.append(null_ratio)

    # Média percentual de nulos
    null_ratio_med = round(sum(nulls) * 100, 2)

    #  Checks
    passed_checks = 0
    total_checks = 0
    range_score = None

    if range_checks:
        for col_name, valid_fn in range_checks.items():
            total_checks += 1
            invalid_count = df.filter(~valid_fn(col(col_name))).count()
            if invalid_count == 0:
                passed_checks += 1

        range_score = round((passed_checks / total_checks) * 100, 2)

    # -------- Resultado Final --------
    resultado = spark.createDataFrame(
        [(fonte, total_rows, f"{null_ratio_med}%", f"{range_score}%" if range_score is not None else "N/A")],
        ["Fonte", "qtd_linhas", "null_ratio_med", "range_checks"]
    )

    display(resultado)


# Colunas para checks
range_checks_cvm = {
    "SIT": lambda c: c == 'EM FUNCIONAMENTO NORMAL',
    "CNPJ_FUNDO": lambda c: length(regexp_replace(c, r'\D', '')) == 14
}

range_checks_cnpj = {
    "CNPJ": lambda c: length(regexp_replace(c, r'\D', '')) == 14
}

range_checks_uf = {
    "sigla": lambda c: c.isin(["AC","AL","AP","AM","BA","CE","DF","ES","GO","MA","MT","MS",
        "MG","PA","PB","PR","PE","PI","RJ","RN","RS","RO","RR","SC",
        "SP","SE","TO"])
}
qualidadeDados(cvm_bruto, "cvm_bruto", range_checks_cvm)
qualidadeDados(brasil_cnpj, "brasil_cnpj", range_checks_cnpj)
qualidadeDados(brasil_uf, "brasil_uf", range_checks_uf)


Fonte,qtd_linhas,null_ratio_med,range_checks
cvm_bruto,47285,1721.68%,50.0%


Fonte,qtd_linhas,null_ratio_med,range_checks
brasil_cnpj,424,0.0%,100.0%


Fonte,qtd_linhas,null_ratio_med,range_checks
brasil_uf,27,0.0%,100.0%
